In [77]:
!pip install catboost

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import seaborn as sns


import plotly.graph_objects as go # Библиотека Plotly. Модуль "Graph Objects"
import plotly.express as px # Библиотека Plotly. Модуль "Express"


from xgboost import XGBClassifier
from xgboost import plot_importance
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split#  функция разбиения на тренировочную и тестовую выборку
from sklearn.metrics import*
from imblearn.metrics import specificity_score
from sklearn.preprocessing import OrdinalEncoder
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [79]:
data = pd.read_csv('https://raw.githubusercontent.com/STEPANPROTSKY/research-project/refs/heads/main/prad_data/genes_recurrence.csv')
data_ = data.copy()
data_ = data.drop(columns = ['key_0'])
data_['patient.biochemical_recurrence'] = data_['patient.biochemical_recurrence'].apply(lambda x: 1 if x == 'yes' else 0)

In [80]:
data_

,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR,patient.biochemical_recurrence
0,67.9197,0.0000,25.2317,136.0453,45.3141,4827.8527,767.7652,0.5149,0.0,922.7600,...,284.2430,1101.4418,38.1050,692.0700,2303.8105,1488.6715,505.1493,199.2791,0.0000,0
1,17.9448,0.0000,17.9448,169.1662,2.6585,20000.6912,503.7834,1.3292,0.0,921.1659,...,383.4868,907.8734,167.4847,756.3397,2712.3218,854.0391,640.6955,291.7690,0.0000,0
2,17.1029,0.3676,1.8382,159.0037,1.1029,4906.4963,251.1029,0.3676,0.0,801.4706,...,598.8971,1727.5735,44.4853,1190.8088,1534.5588,2081.2500,1373.5294,36.0294,0.3676,0
3,17.6115,0.0000,3.7651,175.2959,0.9413,8427.4903,464.5252,0.4706,0.0,878.6916,...,421.2260,1016.1195,64.4782,612.7780,3059.1834,1029.7682,1049.0646,21.1790,0.0000,0
4,43.5168,0.0000,0.5038,263.1062,2.5192,8330.8225,458.4960,0.0000,0.0,1017.2566,...,358.7354,994.5837,163.7486,708.4016,2271.8226,837.3851,752.7396,13.6037,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,33.0500,0.0000,2.9902,47.9539,65.3567,8565.8778,289.1927,0.0000,0.0,690.7305,...,400.6835,1269.1158,95.6856,645.8778,1463.9043,1211.8753,550.6194,32.4648,0.0000,0
536,66.2362,0.0000,13.0130,126.2062,68.0681,23998.2482,997.4975,0.0000,0.0,664.6647,...,462.9630,1486.4865,75.0751,1006.0060,4285.7858,1448.4484,607.1071,56.5566,0.0000,0
537,15.4093,0.0000,0.0000,131.9862,17.4639,14974.4153,380.0973,0.0000,0.0,831.0776,...,387.2883,939.9703,252.7133,1007.7714,2525.0787,773.5493,740.6761,12.3275,0.0000,0
538,31.1257,0.0000,0.0000,353.2561,31.8174,5047.7607,454.4354,0.0000,0.0,842.4693,...,320.2490,1101.8503,142.4866,805.1185,1455.3000,1340.4807,605.2222,687.5324,0.0000,0


In [83]:
X,y = data_.drop(columns = ['patient.biochemical_recurrence']), data_['patient.biochemical_recurrence']

rus = RandomUnderSampler(random_state=42)
X, y = rus.fit_resample(X, y)

#разбиваем на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

## XGBClassifier

In [109]:
model_XGBCls = XGBClassifier(n_estimators=100, early_stopping_rounds=25)
model_XGBCls.fit(X_train, y_train, eval_set=[(X_val, y_val)])
y_pred = model_XGBCls.predict(X_val)

[0]	validation_0-logloss:0.68876
[1]	validation_0-logloss:0.67233
[2]	validation_0-logloss:0.72661
[3]	validation_0-logloss:0.73311
[4]	validation_0-logloss:0.72638
[5]	validation_0-logloss:0.70299
[6]	validation_0-logloss:0.72288
[7]	validation_0-logloss:0.73955
[8]	validation_0-logloss:0.74924
[9]	validation_0-logloss:0.73681
[10]	validation_0-logloss:0.72577
[11]	validation_0-logloss:0.71777
[12]	validation_0-logloss:0.72698
[13]	validation_0-logloss:0.72532
[14]	validation_0-logloss:0.72045
[15]	validation_0-logloss:0.71735
[16]	validation_0-logloss:0.71859
[17]	validation_0-logloss:0.72026
[18]	validation_0-logloss:0.72259
[19]	validation_0-logloss:0.72136
[20]	validation_0-logloss:0.73695
[21]	validation_0-logloss:0.73175
[22]	validation_0-logloss:0.73164
[23]	validation_0-logloss:0.72432
[24]	validation_0-logloss:0.73337
[25]	validation_0-logloss:0.73804
[26]	validation_0-logloss:0.74492


In [110]:
print(f"Accuracy = {accuracy_score(y_val, y_pred)}")
print(f"Precision = {precision_score(y_val, y_pred)}")
print(f"Recall = {recall_score(y_val, y_pred)}")
print(f"F1 Score = {f1_score(y_val, y_pred)}")
print(f"specificity score = {specificity_score(y_val, y_pred)}")

Accuracy = 0.6756756756756757
Precision = 0.6086956521739131
Recall = 0.8235294117647058
F1 Score = 0.7
specificity score = 0.55


In [111]:
importance = model_XGBCls.get_booster().get_score(importance_type='gain')

#если importance пуст, используем стандартный feature_importances_
if not importance:
    importance = dict(zip(data_.columns.drop('patient.biochemical_recurrence'),
                         model_XGBCls.feature_importances_))

#сортировка по убыванию важности
sorted_importance = {k: v for k, v in sorted(importance.items(), key=lambda item: item[1], reverse=True)}

#фильтрация
positive_importance = {k: v for k, v in sorted_importance.items() if v > 0}

features = list(positive_importance.keys())
values = list(positive_importance.values())


fig = go.Figure()
fig.add_trace(go.Bar(
    x=features,
    y=values,
    marker_color='skyblue'
))

fig.update_layout(
    title='Важность признаков (XGBoost, gain)',
    xaxis_title='Признаки',
    yaxis_title='Важность (gain)',
    xaxis_tickangle=-45,
    height=600,
    width=1000
)

fig.show()

In [112]:
top_positive_XGBClassifier = dict(list(positive_importance.items())[:15])

In [113]:
top_positive_XGBClassifier

{'PGLYRP2': 14.30990982055664,
 'SHISA5': 12.422481536865234,
 'KIAA0562': 11.004274368286133,
 'ANXA10': 10.318748474121094,
 'NOX4': 8.8435697555542,
 'FOXD4L6': 6.660566329956055,
 'GDF9': 6.523218154907227,
 'C6orf97': 6.173889636993408,
 'SETDB1': 5.285774230957031,
 'FBP1': 5.2455644607543945,
 'FLJ44635': 4.466713905334473,
 'LONRF1': 4.140183925628662,
 'ANKLE1': 4.008664608001709,
 'TBC1D9B': 3.976919174194336,
 'ELAVL3': 3.6768646240234375}

## LGBMClassifier


In [104]:
LGBMCls = LGBMClassifier(n_estimators=100, early_stopping_rounds=25)
LGBMCls.fit(X_train, y_train, eval_set=[(X_val, y_val)])
y_pred = LGBMCls.predict(X_val)

[LightGBM] [Warning] early_stopping_round is set=25, early_stopping_rounds=25 will be ignored. Current value: early_stopping_round=25
[LightGBM] [Info] Number of positive: 44, number of negative: 41
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056992 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 526580
[LightGBM] [Info] Number of data points in the train set: 85, number of used features: 18497
[LightGBM] [Warning] early_stopping_round is set=25, early_stopping_rounds=25 will be ignored. Current value: early_stopping_round=25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.517647 -> initscore=0.070618
[LightGBM] [Info] Start training from score 0.070618
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 25 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [105]:
print(f"Accuracy = {accuracy_score(y_val, y_pred)}")
print(f"Precision = {precision_score(y_val, y_pred)}")
print(f"Recall = {recall_score(y_val, y_pred)}")
print(f"F1 Score = {f1_score(y_val, y_pred)}")
print(f"specificity score = {specificity_score(y_val, y_pred)}")

Accuracy = 0.5945945945945946
Precision = 0.5454545454545454
Recall = 0.7058823529411765
F1 Score = 0.6153846153846154
specificity score = 0.5


In [106]:
importance = LGBMCls.feature_importances_

#создание словаря {признак: важность}
feature_importance = dict(zip(
    data_.columns.drop('patient.biochemical_recurrence'),
    importance
))

#сортировка по убыванию важности
sorted_importance = {k: v for k, v in sorted(
    feature_importance.items(),
    key=lambda item: item[1],
    reverse=True
)}

#фильтрация
positive_importance = {k: v for k, v in sorted_importance.items() if v > 0}

features = list(positive_importance.keys())
values = list(positive_importance.values())

fig = go.Figure()
fig.add_trace(go.Bar(
    x=features,
    y=values,
    marker_color='skyblue'
))

fig.update_layout(
    title='Важность признаков (LightGBM, gain)',
    xaxis_title='Признаки',
    yaxis_title='Важность (gain)',
    xaxis_tickangle=-45,
    height=600,
    width=1000
)

fig.show()

In [107]:
top_positive_LGBMCls = dict(list(positive_importance.items())[:15])

In [108]:
top_positive_LGBMCls

{'ANKLE1': np.int32(3),
 'FAM106C': np.int32(3),
 'DHRS12': np.int32(2),
 'TRPA1': np.int32(2),
 'ANO2': np.int32(1),
 'ANXA10': np.int32(1),
 'CDH5': np.int32(1),
 'COMT': np.int32(1),
 'DUSP4': np.int32(1),
 'FAM107B': np.int32(1),
 'FLJ44635': np.int32(1),
 'IRAK1': np.int32(1),
 'KIF19': np.int32(1),
 'LOC389791': np.int32(1),
 'LONRF1': np.int32(1)}

## CatBoostClassifier

In [97]:
model = CatBoostClassifier(iterations=100, early_stopping_rounds=25)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

Learning rate set to 0.029703
0:	learn: 0.6740792	total: 3.2s	remaining: 5m 17s
1:	learn: 0.6554167	total: 4.69s	remaining: 3m 49s
2:	learn: 0.6430591	total: 6.24s	remaining: 3m 21s
3:	learn: 0.6273007	total: 8.66s	remaining: 3m 27s
4:	learn: 0.6077672	total: 10.2s	remaining: 3m 12s
5:	learn: 0.5935841	total: 11.6s	remaining: 3m 2s
6:	learn: 0.5811033	total: 13.1s	remaining: 2m 53s
7:	learn: 0.5640019	total: 14.6s	remaining: 2m 47s
8:	learn: 0.5503270	total: 16.1s	remaining: 2m 42s
9:	learn: 0.5368043	total: 17.5s	remaining: 2m 37s
10:	learn: 0.5229896	total: 19.1s	remaining: 2m 34s
11:	learn: 0.5083132	total: 21.5s	remaining: 2m 37s
12:	learn: 0.4950921	total: 23s	remaining: 2m 33s
13:	learn: 0.4833659	total: 24.5s	remaining: 2m 30s
14:	learn: 0.4720856	total: 26s	remaining: 2m 27s
15:	learn: 0.4653016	total: 27.4s	remaining: 2m 24s
16:	learn: 0.4542207	total: 28.9s	remaining: 2m 21s
17:	learn: 0.4416677	total: 30.4s	remaining: 2m 18s
18:	learn: 0.4313871	total: 32.3s	remaining: 2m 17

In [98]:
print(f"Accuracy = {accuracy_score(y_val, y_pred)}")
print(f"Precision = {precision_score(y_val, y_pred)}")
print(f"Recall = {recall_score(y_val, y_pred)}")
print(f"F1 Score = {f1_score(y_val, y_pred)}")
print(f"specificity score = {specificity_score(y_val, y_pred)}")

Accuracy = 0.6216216216216216
Precision = 0.5652173913043478
Recall = 0.7647058823529411
F1 Score = 0.65
specificity score = 0.5


In [99]:
importance = model.get_feature_importance(type='PredictionValuesChange')

#создание словаря {признак: важность}
feature_importance = dict(zip(
    data_.columns.drop('patient.biochemical_recurrence'),
    importance
))

#сортировыка по убыванию
sorted_importance = {k: v for k, v in sorted(
    feature_importance.items(),
    key=lambda item: item[1],
    reverse=True
)}

#фильтрация
positive_importance = {k: v for k, v in sorted_importance.items() if v > 0}

features = list(positive_importance.keys())
values = list(positive_importance.values())

fig = go.Figure()
fig.add_trace(go.Bar(
    x=features,
    y=values,
    marker_color='skyblue'
))

fig.update_layout(
    title='Важность признаков (CatBoost)',
    xaxis_title='Признаки',
    yaxis_title='Важность (PredictionValuesChange)',
    xaxis_tickangle=-45,
    height=600,
    width=1000
)

fig.show()

In [102]:
top_positive_CatBoostCls = dict(list(positive_importance.items())[:15])

In [103]:
top_positive_CatBoostCls

{'NOX4': np.float64(1.9134106868442302),
 'PGPEP1': np.float64(1.1378867719237227),
 'NKX6-1': np.float64(1.103430550017557),
 'TPTE2P3': np.float64(1.0926869490393782),
 'C3orf36': np.float64(1.09055651679961),
 'C17orf81': np.float64(1.0905263220889838),
 'SRGAP2': np.float64(1.074045602813039),
 'ESAM': np.float64(1.0057974102964866),
 'TEX10': np.float64(0.9842447724618434),
 'KIF19': np.float64(0.9751559783771984),
 'CEP72': np.float64(0.9703589265304049),
 'GSDMA': np.float64(0.9695560074280839),
 'CSF2RA': np.float64(0.9321541823413456),
 'ELTD1': np.float64(0.8802503927837088),
 'ENG': np.float64(0.8626840927557399)}

## Результаты


In [114]:
dicts = [top_positive_CatBoostCls, top_positive_LGBMCls, top_positive_CatBoostCls]
common_keys = set.intersection(*(set(d.keys()) for d in dicts))

In [116]:
dicts

[{'NOX4': np.float64(1.9134106868442302),
  'PGPEP1': np.float64(1.1378867719237227),
  'NKX6-1': np.float64(1.103430550017557),
  'TPTE2P3': np.float64(1.0926869490393782),
  'C3orf36': np.float64(1.09055651679961),
  'C17orf81': np.float64(1.0905263220889838),
  'SRGAP2': np.float64(1.074045602813039),
  'ESAM': np.float64(1.0057974102964866),
  'TEX10': np.float64(0.9842447724618434),
  'KIF19': np.float64(0.9751559783771984),
  'CEP72': np.float64(0.9703589265304049),
  'GSDMA': np.float64(0.9695560074280839),
  'CSF2RA': np.float64(0.9321541823413456),
  'ELTD1': np.float64(0.8802503927837088),
  'ENG': np.float64(0.8626840927557399)},
 {'ANKLE1': np.int32(3),
  'FAM106C': np.int32(3),
  'DHRS12': np.int32(2),
  'TRPA1': np.int32(2),
  'ANO2': np.int32(1),
  'ANXA10': np.int32(1),
  'CDH5': np.int32(1),
  'COMT': np.int32(1),
  'DUSP4': np.int32(1),
  'FAM107B': np.int32(1),
  'FLJ44635': np.int32(1),
  'IRAK1': np.int32(1),
  'KIF19': np.int32(1),
  'LOC389791': np.int32(1),
  'L

### Пересечения результатов моделей

In [115]:
common_keys

{'KIF19'}

### Кол-во встреч генов исходя из результатов моделей

In [118]:
all_keys = [key for d in dicts for key in d.keys()]

key_counts = Counter(all_keys)

print(key_counts)

Counter({'KIF19': 3, 'NOX4': 2, 'PGPEP1': 2, 'NKX6-1': 2, 'TPTE2P3': 2, 'C3orf36': 2, 'C17orf81': 2, 'SRGAP2': 2, 'ESAM': 2, 'TEX10': 2, 'CEP72': 2, 'GSDMA': 2, 'CSF2RA': 2, 'ELTD1': 2, 'ENG': 2, 'ANKLE1': 1, 'FAM106C': 1, 'DHRS12': 1, 'TRPA1': 1, 'ANO2': 1, 'ANXA10': 1, 'CDH5': 1, 'COMT': 1, 'DUSP4': 1, 'FAM107B': 1, 'FLJ44635': 1, 'IRAK1': 1, 'LOC389791': 1, 'LONRF1': 1})
